Ce TD nécessite l'installation de la librairie PyTorch qui offre la fonctionnalité de différentiation automatique (et de faço optionnelle, le calcul sur GPU). Aucune manipulation n'est requise sur Google Colab, qui reste la solution la plus pratique. Si le TP est exécuté sur une machine locale avec la distribution Anaconda, il convient d'installer la librairie au préalable dans le console "Anaconda Prompt" à l'aide de la commande :
```
conda install pytorch torchvision torchaudio cudatoolkit=11.6 -c pytorch -c conda-forge
```

In [2]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
import time
import random
torch.set_default_tensor_type(torch.DoubleTensor)

On charge notre jeu de données

In [3]:
data = pd.read_csv('https://joon-kwon.github.io/optim-mathsv/blogData_train.csv', header=None)
X = torch.from_numpy(data.values[:, :-1])
y = torch.from_numpy(data.values[:, -1].reshape(-1))
d = len(X[0])
n = len(X)

On définit notre fonction objectif qui correspond à une régression linéaire aux moindres carrés. On peut éventuellement passer en second argument un mini-batch (sous la forme d'une liste contenant les indices correspondants), si on souhaite ne calculer la valeur que sur ce mini-batch.

In [4]:
def f(x, minibatch=None):
    if minibatch is None:
        return torch.mean((torch.tensordot(X, x, dims=([1], [0])) - y)**2)
    else:
        return torch.mean((torch.tensordot(X[minibatch], x, dims=([1], [0])) - y[minibatch])**2)

La différentiation automatique sous PyTorch fonctionne de la façon suivante.

In [5]:
x = torch.zeros(d) # Le point en lequel on souhaite calculer un gradient
x.requires_grad = True # On indique qu'on va calculer un gradient par rapport à cette variable
value = f(x) # On calcule d'abord la fonction elle-même en ce point
value.backward() # On demande à pytorch de calculer le gradient
print(x.grad) # Le gradient correspondant est alors stocké dans x.grad
x = x.detach() # On arrête la fonctionnalité de différentiation automatique (attention à sauver x.grad auparavant)

tensor([-3.4303e+03, -2.6301e+03, -3.2311e+01, -1.6469e+04, -2.9147e+03,
        -1.4161e+03, -1.6398e+03, -3.8449e-01, -1.1305e+04, -9.8679e+02,
        -1.2496e+03, -1.6925e+03,  0.0000e+00, -1.1273e+04, -5.8779e+02,
        -2.8543e+03, -2.0955e+03, -3.2311e+01, -1.2334e+04, -2.5479e+03,
        -1.6657e+02, -2.4195e+03,  8.7663e+03, -1.1119e+04,  7.3602e+00,
        -3.0588e+01, -2.9762e+01, -2.2749e-02, -2.0534e+02, -2.5787e+01,
        -1.1696e+01, -2.0189e+01,  0.0000e+00, -1.5924e+02, -5.0265e+00,
        -1.1143e+01, -2.0311e+01,  0.0000e+00, -1.5923e+02, -8.3974e-03,
        -2.8544e+01, -2.8059e+01, -2.2749e-02, -1.8192e+02, -1.9055e+01,
        -5.5318e-01, -3.0101e+01,  1.3194e+02, -1.5385e+02, -7.3859e-03,
        -3.1678e+03, -2.2541e+03, -6.9056e+02, -2.7200e+03, -1.5636e+03,
        -2.9403e+01, -1.9855e+01, -6.8439e+00, -2.8239e+01, -1.3012e+01,
        -2.3238e+02, -5.2429e+04, -3.5727e-02, -3.3457e+00,  0.0000e+00,
        -3.7788e-03, -7.0982e+00, -2.7347e+00, -5.2

*Question 1*: Utiliser la différentiation automatique pour implémenter une descente de gradient (à pas constant). Faire cesser les itérations au bout de 5 secondes (par exemple). Afficher la valeur de la fonction objectif en la dernière itérée. Essayer par tâtonnements différentes valeurs pour le pas.

On peut par exemple se donner un mini-batch de taille 100 de la façon suivante.

In [6]:
minibatch = random.sample(range(0,n), 100)
print(minibatch)

[5637, 18997, 1809, 16409, 8448, 31281, 45607, 37121, 41448, 15953, 44146, 11833, 31044, 1068, 8178, 20351, 1919, 43602, 34816, 26528, 14434, 4838, 12740, 14288, 37090, 13234, 49911, 37147, 33516, 41755, 38279, 11431, 41757, 583, 43563, 48877, 33311, 10767, 45656, 17079, 18506, 39317, 5340, 10714, 36794, 38491, 20928, 30530, 51302, 27852, 7782, 23155, 1646, 15544, 3547, 10580, 41454, 25784, 46728, 17925, 17197, 3800, 47644, 18723, 38550, 27994, 2940, 39863, 40880, 21318, 2456, 3941, 28862, 8424, 47989, 50948, 46165, 11791, 46105, 38226, 38584, 14130, 38902, 49385, 21184, 44699, 4783, 51564, 38397, 13413, 40951, 32188, 3190, 23022, 52305, 12645, 6048, 10623, 15907, 43684]


*Question 2*: Modifier le code de la question 1 pour utiliser des mini-batchs. Essayer différentes tailles de mini-batchs. Commenter les résultats.

PyTorch propose des implémentations d'algorithmes d'optimisation qui peuvent être utilisés de la façon suivante.
`lr` correspond au pas. La prodécure `.step()` opère l'itération de l'algorithme d'optimisation en utilisant le gradient qui vient d'être calculé, et met à jour la valeur de la variable `x`.

In [7]:
x = torch.zeros(d)
x.requires_grad = True
optimizer = torch.optim.SGD([x], lr=1e-9)
for t in range(100):
    optimizer.zero_grad()
    value = f(x)
    value.backward()
    optimizer.step()
f(x)

tensor(1377.3830, grad_fn=<MeanBackward0>)

*Question 3*: À la place de SGD (qui correspond tout simplement à la descente de gradient), utiliser les implémentations de RMSprop, Adam, etc. (voir https://pytorch.org/docs/stable/optim.html), éventuellement avec des mini-batchs, toujours en stoppant l'exécution au bout de 5 secondes, et en affichant la valeur de la fonction objectif en la dernière itérée. Comparer les résultats obtenus.